# 데이터 불러오기

In [12]:
import pandas as pd
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
#customer_21 = pd.read_csv(DATA_PATH_2 + 'customer_21.csv', encoding="euc-kr")
customer_22 = pd.read_csv(DATA_PATH_2 + 'customer_22.csv', encoding="euc-kr")
#customer_23 = pd.read_csv(DATA_PATH_2 + 'customer_23.csv', encoding="euc-kr")

# 필요 라이브러리

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
#from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import random

#from ggplot import *
#from scipy.stats import logistic
#from sklearn.metrics import roc_curve
#from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix, mean_squared_error
#from sklearn.model_selection import train_test_split
#from sklearn import preprocessing
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import *
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
#import graphviz
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.svm import SVC, SVR

# 한글 폰트 실행
from matplotlib import rcParams
from matplotlib import font_manager,rc
rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='NanumGothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)


# 분석에 도움주는 주요 기본 함수 정의
## 메모리 줄여주는 함수 (일반)

## 시간체크함수

In [3]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Execution time for {func.__name__}: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
        return result
    return wrapper

In [2]:
import pandas as pd 
import numpy as np

def reduce_mem_usage(df, float16_as32=True):
    # memory_usage()는 데이터프레임의 각 열의 메모리 사용량을 계산하고, sum은 그 값을 합산합니다. B -> KB -> MB로 변환합니다.
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터프레임의 초기 메모리 사용량은 {:.2f} MB입니다.'.format(start_mem))

    for col in df.columns:  # 각 열의 이름을 반복합니다.
        col_type = df[col].dtype  # 열의 데이터 타입을 가져옵니다.
        if col_type != object and str(col_type) != 'category':  # 객체 타입이 아닌 수치형 변수만 처리합니다.
            c_min, c_max = df[col].min(), df[col].max()  # 해당 열의 최소값과 최대값을 계산합니다.
            if str(col_type)[:3] == 'int':  # 정수형 변수일 경우 (int8, int16, int32, int64 모두 포함)
                # 이 열의 값 범위가 int8의 범위에 있을 경우 타입을 int8로 변환합니다. (-128부터 127까지)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                # 이 열의 값 범위가 int16의 범위에 있을 경우 타입을 int16으로 변환합니다. (-32,768부터 32,767까지)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                # 이 열의 값 범위가 int32의 범위에 있을 경우 타입을 int32로 변환합니다. (-2,147,483,648부터 2,147,483,647까지)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                # 이 열의 값 범위가 int64의 범위에 있을 경우 타입을 int64로 변환합니다. (-9,223,372,036,854,775,808부터 9,223,372,036,854,775,807까지)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:  # 실수형일 경우
                # 값이 float16의 범위에 있을 경우 타입을 변환합니다. 더 높은 정밀도가 필요하면 float32를 고려합니다.
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:  # 더 높은 정밀도가 필요하면 float32를 선택할 수 있습니다.
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)
                # 값이 float32의 범위에 있을 경우 타입을 float32로 변환합니다.
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                # 값이 float64의 범위에 있을 경우 타입을 float64로 변환합니다.
                else:
                    df[col] = df[col].astype(np.float64)
    # 최종 메모리 사용량을 계산합니다.
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 데이터프레임의 메모리 사용량은: {:.2f} MB입니다.'.format(end_mem))
    # 초기 메모리 사용량과 비교하여 몇 퍼센트 감소했는지 계산합니다.
    print('메모리 사용량이 {:.1f}% 감소했습니다.'.format(100 * (start_mem - end_mem) / start_mem))

    return df

## 데이터 타입 문자열로 바꿔주는 함수

In [38]:
@execution_time
def clean_df(data):
    data['고객ID'] = data['고객ID'].astype(str)
    data = reduce_mem_usage(data)
    print(data.info())
    return data

In [39]:
customer_22=clean_df(customer_22)

데이터프레임의 초기 메모리 사용량은 2140.22 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 1564.01 MB입니다.
메모리 사용량이 26.9% 감소했습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21578686 entries, 0 to 21578685
Data columns (total 13 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   기준년월      int32 
 1   고객ID      object
 2   연령대       object
 3   성별        object
 4   고객등급      object
 5   자택_시도     object
 6   자택_시군구    object
 7   수신_요구불예금  int32 
 8   수신_거치식예금  int32 
 9   수신_적립식예금  int32 
 10  수신_펀드     int32 
 11  수신_외화예금   int32 
 12  대출금액      int32 
dtypes: int32(7), object(6)
memory usage: 1.5+ GB
None
Execution time for clean_df: 0h 0m 8.28s


## 고객연령대 나누는 함수

In [47]:
def customer_generation(df, feature): # feature에 뽑고 싶은 연령대를 입력한다. "연령대"
    df = df[df['연령대'].isin([f'{feature}'])]
    print(f'{feature}만 추출한 데이터프레임입니다.')
    print(df.head())
    return df

In [48]:
customer_22_40=customer_generation(customer_22,"40대")

40대만 추출한 데이터프레임입니다.
       기준년월        고객ID  연령대 성별  고객등급  자택_시도 자택_시군구  수신_요구불예금  수신_거치식예금  \
712  202212  9997377777  40대  남  일반등급  서울특별시    강북구         0         0   
713  202212  9997372016  40대  남  일반등급  대구광역시     동구   7600000         0   
714  202212  9997367955  40대  여  일반등급   경상북도    영주시    200000         0   
715  202212  9997366851  40대  여  일반등급  대구광역시    수성구    300000         0   
716  202212  9997365476  40대  여  일반등급  대구광역시     중구         0         0   

     수신_적립식예금  수신_펀드  수신_외화예금  대출금액  
712         0      0        0     0  
713         0      0        0     0  
714         0      0        0     0  
715         0      0        0     0  
716         0      0        0     0  


In [24]:
type(customer_22_40)

pandas.core.frame.DataFrame

## 고객연령대 나눈 후, ID 중복없이 추출함수

In [22]:
def customer_generation_id(df):
    selected_df = df.loc[:, ['고객ID', '연령대']]
    df_edit = pd.DataFrame(selected_df['고객ID'].unique(), columns=['고객ID'])
    print(df_edit.head())
    return df_edit

In [25]:
customer_22_40_id = customer_generation_id(customer_22_40)

         고객ID
0  9997377777
1  9997372016
2  9997367955
3  9997366851
4  9997365476


In [26]:
type(customer_22_40_id)

pandas.core.frame.DataFrame

## 고객 연령대 나누고, 바로 id를 DataFrame으로 추출하는 함수
- feature에 연령대 명을 정확히 입력하면, 해댱하는 id를 바로 추출할 수 있다

In [32]:
def customer_gen_id(df, feature=None): # feature에 뽑고 싶은 연령대를 입력한다. 예: "60대이상", "40대"
    df_filtered = df[df['연령대'].isin([f'{feature}'])]
    selected_df = df_filtered.loc[:, ['고객ID', '연령대']]
    df_edit = pd.DataFrame(selected_df['고객ID'].unique(), columns=['고객ID'])
    print(df_edit.head())
    return df_edit

In [34]:
cu_22_40=customer_gen_id(customer_22, "40대")

         고객ID
0  9997377777
1  9997372016
2  9997367955
3  9997366851
4  9997365476


In [35]:
type(cu_22_40)

pandas.core.frame.DataFrame

## 가맹업종명 재분류하는 함수

In [36]:
@execution_time
def rename_category(df):
    mapping = {
        '교육/학원': ['보습학원', '예·체능계학원', '외국어학원', '기타 교육기관', '기능학원', '학원', '학습지교육', '초중고교육기관'],
        '미용': ['미용원', '피부미용실'],
        '스포츠/레저': ['기타레져업소', '스포츠·레져용품', '스포츠의류', '스크린골프', '헬스클럽', '골프경기장', '종합레져타운', '골프용품 전문점', '골프연습장', '안마/스포츠마사지', '레져업소', '수 영 장'],
        '여행/교통': ['기타숙박업', '철도', '특급호텔', '1급 호텔', '렌터카', '항공사', '콘도', '2급 호텔', '관광여행', '여객선', '고속버스', '택시'],
        '외식': ['일반한식', '주점', '서양음식', '중국식', '일식·회집', '한정식', '칵테일바', '갈비전문점'],
        '대형마트': ['슈퍼마켓', '농협하나로클럽', '농,축협직영매장', '대형할인점'],
        '백화점': ['면 세 점', '자사카드발행백화점'],
        '편의점': ['편 의 점'],
        '음식료품': ['정육점', '스넥', '농·축·수산품', '농축수산가공품', '주류판매점', '홍삼제품', '인삼제품', '기타건강식품'],
        '카페/베이커리': ['기타음료식품', '제과점'],
        '의료/보험': ['약 국', '의 원', '한 의 원', '치과의원', '기타의료기관 및 기타의료기기', '한약방', '한방병원', '생명보험', '건강진단', '기타보험', '병 원(응급실운영)', '치과병원', '동물병원', '종합병원', '손해보험'],
        '주유': ['SK주유소', '쌍용S-OIL', 'GS주유소', '주유소', 'SK가스충전소', '현대정유(오일뱅크)', 'GS가스충전소', '쌍용S-OIL가스충전소', '현대정유가스충전소', 'E1가스충전소', '전기차충전소'],
        '문화생활': ['문화취미기타', '영화관', '티켓'],
        '이동통신요금': ['이동통신요금'],
        '반려동물': ['애완동물'],
        '간편결제': ['인터넷P/G'],
        '기타': ['전자상거래상품권전문판매', '캐주얼의류', 'CATV', '택시회사', '위성방송', '음반영상물', '제 화','주방용구', '연 쇄 점', '액세서리', '전문서적', '주차장', '민예·공예품', '건축용 요업제품', '단란주점', '옷감·직물', '자동차시트·타이어', '독서실', '농기계', '조명기구', '세차장', '사무서비스', '컴퓨터', '가 방', '보일러·펌프·샷시', '카페트,커튼,천막,지물', '자동차부품', '자동차정비', '기타유통업', '이륜차판매', '카인테리어', '화물운송', '내의판매', '목재·석재·철물', '기타 사무용품', '기타가구', '신 발', '비영리/비대상', '기타대인서비스', '기타전기제품', 'DP&E', '주방용식기', '가타자동차서비스', '기타농업관련', '구내매점(국가기관등)', '아동의류', '이용원', '조세서비스', '기념품점', '유흥주점', '통신기기', '화원', '화장품', '기타잡화', '유아원', 'LPG취급점', '페인트', '출판 및 인쇄물', '귀금속', '문구용품', '위탁급식업', '비료,사료,종자', '침구·수예점', '당구장', '가전제품', '인터넷Mall', '가정용품수리', '사무·통신기기수리', '안경', '세탁소', '사우나', '일반서적', '노 래 방', '기계공구',  '완구점', '기타의류', '미곡상', '기타용역서비스', '기타수리서비스', '기타건축자재', '일반가구', '사무용 OA기기', '인테리어전문', '가례서비스업', '시 계', '비영리/대상', '윤활유전문판매', '정수기', '공공요금대행서비스/소득공제비대상', '화방·표구점', '볼 링 장', '인터넷종합Mall', '기타 전문점', '통신판매업1', '종합용역', '전자상거래상품권', '보관및 창고업', '유류판매', '업종미등록', '통신서비스/소득공제비대상', '기타서적문구', '수족관', '공공요금대행서비스/소득공제대상', '상품권전문판매', '혼례서비스업', '냉열기기', '비씨카드 정산용(할인)', '일반(통신판매)', '기타4', '기타1', '부동산중개·임대', 'CATV홈쇼핑', '소프트웨어', '의료용품', '정장', '맞춤복점', '단체복', '상 품 권', 'PG상품권', '비씨카드 정산용가맹점',  '미용재료']
    }
    for category, values in mapping.items():
        df['가맹점업종명'] = df['가맹점업종명'].replace(values, category)
    print(df['가맹점업종명'].unique())
    return df

In [37]:
import pandas as pd
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
#customer_21 = pd.read_csv(DATA_PATH_2 + 'customer_21.csv', encoding="euc-kr")
customer_22 = pd.read_csv(DATA_PATH_2 + 'customer_22.csv', encoding="euc-kr")
#customer_23 = pd.read_csv(DATA_PATH_2 + 'customer_23.csv', encoding="euc-kr")
card_22 = pd.read_csv(DATA_PATH + 'card_22.csv', encoding="euc-kr")

ca_22 = rename_category(card_22)

['외식' '기타' '미용' '스포츠/레저' '의료/보험' '여행/교통' '카페/베이커리' '주유' '대형마트' '음식료품'
 '편의점' '교육/학원' '문화생활' '간편결제' '반려동물' '백화점' '이동통신요금']


# 정제한 card, customer 합치는 함수

In [53]:
def join(df1, df2, key, way): # merge할 두 데이터를 df1, df2에 쓴다. key에 join 할 컬럼 이름 쓴다. way에는 조인 방식 쓴다.
    df_joined= pd.merge(df1, df2, on=f'{key}', how=f'{way}')
    print(type(df_joined))
    print(df_joined.head(3))
    #card_customer_22_40_id.to_csv('cc_22_40.csv', index=False, encoding='utf-8-sig')
    return df_joined

In [54]:
cc_22_40 = join(ca_22, cu_22_40, "고객ID", "inner")

<class 'pandas.core.frame.DataFrame'>
     거래년월        고객ID 가맹점업종명 가맹점_광역시도 가맹점_시군구  승인건수   승인금액
0  202212  5946674163     외식     경상북도     영천시  5번이하  80000
1  202212  7605672934     외식     경상북도     영천시  5번이하  60000
2  202212  7652171123     외식     경상북도     영천시  5번이하  50000


## 원하는 연령대, 컬럼들만 뽑아 dataframe으로 만드는 함수

In [55]:
def slicing(df,generation=None,feature=None): # feature에 뽑고 싶은 연령대를 입력한다. 예: "60대이상", "40대"
    df_filtered = df[df['연령대'].isin([f'{generation}'])]
    selected_df = df_filtered.loc[:, [f'[{feature}', '연령대']]
    df_edit = pd.DataFrame(selected_df['고객ID'].unique(), columns=['고객ID'])
    print(type(df_edit))
    print(df_edit.head())
    return df_edit

In [ ]:
slicing(customer_22, "40대", "고")s

In [ ]:
def slicing(df, generation=None, feature=None):  
    # generation과 feature가 단일 값일 경우 리스트로 변환
    if isinstance(generation, str):  
        generation = [generation]
    if isinstance(feature, str):  
        feature = [feature]
    
    # generation 필터링 (여러 값을 받을 수 있도록 isin 사용)
    df_filtered = df[df['연령대'].isin(generation)]
    
    # feature가 여러 개일 수 있으므로 isin을 사용하여 필터링
    selected_df = df_filtered.loc[:, feature + ['연령대']]  # feature 리스트와 '연령대' 추가

    # 고객ID의 고유값만 선택하여 DataFrame 생성
    df_edit = pd.DataFrame(selected_df['고객ID'].unique(), columns=['고객ID'])
    
    print(type(df_edit))
    print(df_edit.head())
    return df_edit

# 함수 호출 예제
# generation과 feature에 여러 값을 리스트로 전달
df_result = slicing(customer_22, generation=["40대", "50대"], feature=["성별", "고객등급"])
print(df_result.head())
